In [ ]:
"""Functions for plugging into Pipulate-frameworks for conducting SEO investigations."""

In [ ]:
import requests, re
from collections import namedtuple
Response = namedtuple('Response', 'ok status_code text')
flags = re.DOTALL

In [ ]:
def Title(**row_dict):
    pattern = re.compile(pattern=r'<title\s?>(.*?)</title\s?>', flags=flags)
    matches = pattern.findall(string=row_dict['response'].text)
    if matches:
        text = matches[0].strip()
        return Response(ok=True, status_code='200', text=text)
    else:
        return Response(ok=True, status_code='200', text=None)

In [ ]:
if __name__ == '__main__':
    test_url = 'http://mikelev.in/'
    response = requests.get(test_url)
    print(Title(url=test_url, response=response))